## Setup

To complete the following guide you will need to install the following packages:
- fireworks-ai
- pandas
- requests

You will also need:

- Fireworks account (https://fireworks.ai/)
- Fireworks API key
- The firectl command-line interface (https://docs.fireworks.ai/tools-sdks/firectl/firectl)

In [1]:
!pip install pandas requests fireworks-ai --quiet

You should consider upgrading via the '/Users/scottkramer/.pyenv/versions/3.8.16/envs/fine-tuning-workshop/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import json
import os
import time

from fireworks.client import Fireworks
import pandas as pd
import requests

In [3]:
# Sign-in to your Fireworks account
!firectl signin

2024/11/18 17:40:06 There are updates available.
Current version: 1.2.0
Latest version: 1.4.2

To upgrade to the latest version, run
  $ sudo firectl upgrade

Signed in as: sdkramer10@gmail.com
Account ID: sdkramer10-5e98cb


In [3]:
# Make sure you have the FIREWORKS_API_KEY environment variable set to your account's key!
# os.environ['FIREWORKS_API_KEY'] = 'XXX'

client = Fireworks()

# Replace the line below with your Fireworks account id
account_id = 'XXX'

## Problem Definition: Career Coach Chatbot

*Note: The data used in this example were synthetically generated with Claude 3.5 Sonnet*

This example provides a template for fine-tuning chatbots. This notebook focuses on fine-tuning a chatbot to act as a career coach for software developers.

### Data

The data can be found in the `data` folder.

We will use the following datasets:
- `./data/training_data.csv`
- `./data/test_data.csv`

Each of these datasets consists of conversations between a software developer and a career coach.

In [41]:
training_df = pd.read_csv("data/training_conversations.csv")
test_df = pd.read_csv("data/test_conversations.csv")

In [69]:
system_msg = """You are a technical career coach for software developers. Your role is to help developers explore their career options and make informed decisions.
Core principles:

Ask thoughtful, open-ended questions focused on understanding their situation
Guide developers to insights rather than giving direct advice
Consider both technical growth and personal fulfillment
Balance immediate needs with long-term career development
Show understanding of software development roles, transitions, and industry trends

Maintain a supportive, professional tone while helping developers think deeply about their career decisions. Focus on understanding their unique context and goals rather than providing generic advice."""

In [17]:
# Converts the training examples to the format expected by Fireworks.
training_rows = list()
for _, group in training_df.groupby("conversation_name"):
    messages = [{"role": "system", "content": system_msg}]

    for _, row in group.iterrows():
        role = "user" if row["user"] == "Developer" else "assistant"
        messages.append({"role": role, "content": row["message"]})
    
    training_row = {"messages": messages}   
    training_rows.append(training_row)

In [21]:
# Writes the data to a file so that it can be uploaded to Fireworks
dataset_file_name = 'career-coach-chatbot-training-dataset.jsonl'
dataset_id = 'career-coach-chatbot-training-dataset-v1'

with open(dataset_file_name, 'w') as f:
    for obj in training_rows:
        json.dump(obj, f)
        f.write('\n')

In [71]:
# Follow instructions here to first install the firectil CLI - https://readme.fireworks.ai/docs/fine-tuning-models#installing-firectl
# Then run this command to upload the file to Fireworks
!firectl create dataset {dataset_id} {dataset_file_name}

### Fine-tune model
We now fine-tune the model with the generated dataset. For this example, we set the batch size smaller than the default, as each example in the batch contains an entire conversation, and is effectively multiple examples.

In [72]:
# Creates a training job with the default hyperparameters
!firectl create fine-tuning-job --settings-file fireworks-fine-tune-config.yaml --display-name "developer career coach" --dataset {dataset_id} 

In [34]:
# NOTE THAT THESE IDS WILL CHANGE WHEN YOU RUN THE FINE-TUNING JOB ON YOUR ACCOUNT!!!
# The model id is printed in the stdout of the cell above as Name: accounts/{account_id}/fineTuningJobs/{model_id}
model_id = 'c30a1bea220441c18348919618b09601'

In [76]:
# Wait until the State of the two fine-tuning jobs are listed as COMPLETED (~10-20 minutes)
!firectl get fine-tuning-job {model_id}

In [37]:
# Deploy the fine-tuned model
!firectl deploy {model_id}

2024/11/22 17:19:39 There are updates available.
Current version: 1.2.0
Latest version: 1.4.3

To upgrade to the latest version, run
  $ sudo firectl upgrade



In [75]:
# Wait until the the Deployed Model Refs lists the state of the model as "DEPLOYED" (~5-20 minutes).
!firectl get model {model_id}

### Evaluation
We now evaluate the fine-tuned model on our test data. For each message from the coach in the test data, we track the generated response, the response from the golden dataset, and the conversation history context.

We can then use techniques such as an LLM judge or heuristics to evaluate the results.

In [50]:
def generate_response(model, messages):
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

In [66]:
model = f'accounts/{account_id}/models/{model_id}'

result_rows = []
for _, group in test_df.groupby("conversation_name"):
    messages = [{"role": "system", "content": system_msg}]

    for row_idx, row in group.iterrows():
        role = "user" if row["user"] == "Developer" else "assistant"

        if role == "user":
            messages.append({"role": role, "content": row["message"]})
            continue

        ideal_response = row["message"]
        actual_response = generate_response(model, messages)
        conversation_history = "\n".join(
            f'{message["role"].capitalize()}: {message["content"]}'
            for message in messages if message["role"] != "system"
        )
        
        result_rows.append({
            "conversation": conversation_history,
            "ideal_response": ideal_response,
            "actual_response": actual_response,
        })

        messages.append({"role": role, "content": row["message"]})

results_df = pd.DataFrame(result_rows)

In [68]:
results_df.to_csv("evaluations.csv", index=False)

In [74]:
!firectl undeploy {model_id}

2024/11/22 17:57:14 There are updates available.
Current version: 1.2.0
Latest version: 1.4.3

To upgrade to the latest version, run
  $ sudo firectl upgrade

2024/11/22 17:57:16 Failed to execute: rpc error: code = Unimplemented desc = unknown method UndeployModel for service gateway.Gateway
